<a href="https://colab.research.google.com/github/Vac1k/Aurora_Project/blob/main/Aurora_Predict_model_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт бібліотек

In [2]:
import difflib
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.frequent_patterns import apriori, association_rules
from docx import Document

Завантаження та обробка даних про товари

In [3]:
file_path_inventory = "/content/Залишки по ТТ.xlsx"
inventory_df = pd.read_excel(file_path_inventory)

In [4]:
inventory_df = inventory_df.rename(columns={
    "Рівень категорії 3": "Category_Level3",
    "Назва товару": "Product_Full_Name",
    "К-сть залишків на кінець дня": "Stock_Quantity_End_Day",
    "Собівартість залишків на кінець дня": "Stock_Cost_End_Day"
})

In [5]:
inventory_df["Product_ID"] = inventory_df["Product_Full_Name"].str.extract(r'^(\d+)')
inventory_df["Product_Name"] = inventory_df["Product_Full_Name"].str.replace(r'^\d+\s+', '', regex=True)
inventory_df["Product_ID"] = pd.to_numeric(inventory_df["Product_ID"], errors='coerce')
inventory_df["Calculated_Price"] = (inventory_df["Stock_Cost_End_Day"] / inventory_df["Stock_Quantity_End_Day"]).round(3)

Завантаження подій та їх аналіз

In [6]:
def load_events_from_docx():
    doc = Document("/content/12123.docx")
    events_dict, current_event = {}, None

    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        if not text:
            continue
        if text[0].isdigit() and "." in text:
            current_event = text.split(". ", 1)[1]
            events_dict[current_event] = []
        elif current_event:
            events_dict[current_event].append(text)
    return events_dict

events_data = load_events_from_docx()

In [7]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
inventory_df["Product_Vector"] = inventory_df["Product_Name"].apply(lambda x: model.encode(x))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Векторизація товарів для AI-пошуку

In [8]:
df1 = pd.read_excel("товари у чеку_1.xlsx")
basket = df1.groupby(["Номер чеку", "Назва товару"]).size().unstack(fill_value=0)
basket[basket > 1] = 1

In [9]:
# 🧠 Apriori + правила
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


Фільтрація товарів за подією

In [10]:


def filter_by_event(event_name):
    products = events_data.get(event_name, [])
    matched_rows = []

    for product in products:
        match = difflib.get_close_matches(product, inventory_df["Product_Name"], n=1, cutoff=0.6)
        if match:
            row = inventory_df[inventory_df["Product_Name"] == match[0]]
            if not row.empty:
                matched_rows.append(row)

    if matched_rows:
        return pd.concat(matched_rows)[["Product_Name", "Calculated_Price"]]
    else:
        return pd.DataFrame(columns=["Product_Name", "Calculated_Price"])

In [11]:
def recommend_products(query, top_n=5):
    q_vec = model.encode(query)
    inventory_df["Similarity"] = inventory_df["Product_Vector"].apply(lambda vec: cosine_similarity([q_vec], [vec])[0][0])
    return inventory_df.sort_values("Similarity", ascending=False).head(top_n)[["Product_Name", "Calculated_Price", "Similarity"]]

In [12]:
def recommend_by_product(product_name):
    if rules.empty or 'consequents' not in rules.columns:
        print("⚠️ Немає асоціативних правил або колонка 'consequents' відсутня.")
        return recommend_products(product_name, top_n=5)

    related = rules[rules["antecedents"].apply(lambda x: product_name in list(x))]

    if related.empty:
        print("⚠️ Немає супутніх покупок для цього товару.")
        return recommend_products(product_name, top_n=5)

    consequents = related["consequents"].explode().value_counts().head(5).index
    return inventory_df[inventory_df["Product_Name"].isin(consequents)][["Product_Name", "Calculated_Price"]]

In [17]:
def process_user_query(query):
    query = query.lower().strip()

    # 🔍 Спочатку AI-векторизація
    initial = recommend_products(query, top_n=1)
    if initial.empty:
        print("⚠️ Не знайдено релевантних товарів.")
        return pd.DataFrame()

    base_product = initial.iloc[0]["Product_Name"]
    print(f"🛍 Найближчий товар: {base_product}")

    # 🎯 Додатково: якщо виявлено подію, додамо товари події до списку
    matched_event = next((event for event in events_data if event.lower() in query), None)
    if matched_event:
        print(f"🎯 Також виявлено подію: {matched_event}")
        event_items = filter_by_event(matched_event)
    else:
        event_items = pd.DataFrame()

    # 🧠 Супутні покупки
    if rules.empty or 'consequents' not in rules.columns:
        print("⚠️ Немає асоціативних правил. Повертаю AI-рекомендації.")
        ai_result = recommend_products(query, top_n=5)
    else:
        related = rules[rules["antecedents"].apply(lambda x: base_product in list(x))]
        if related.empty:
            print("⚠️ Немає супутніх покупок. Повертаю AI-рекомендації.")
            ai_result = recommend_products(query, top_n=5)
        else:
            consequents = related["consequents"].explode().value_counts().head(5).index
            ai_result = inventory_df[inventory_df["Product_Name"].isin(consequents)][["Product_Name", "Calculated_Price"]]

    # 🔗 Об’єднуємо результати (AI + події, якщо є)
    final_result = pd.concat([ai_result, event_items]).drop_duplicates(subset=["Product_Name"]).head(5)

    return final_result

AI-пошук товарів

Speech Recognition

In [18]:
def recognize_speech():
    recognizer = sr.Recognizer()
    try:
        with sr.Microphone() as source:
            audio = recognizer.listen(source)
            return recognizer.recognize_google(audio, language="uk-UA")
    except (sr.UnknownValueError, sr.RequestError, OSError):
        return None

Основна логіка програми

In [ ]:
# 🧾 Інтерактивний ввід запиту користувачем
def run_assistant():
    print("Введи 'вихід', щоб завершити.\n")

    while True:
        user_input = input("📝 Ваш запит: ")
        if user_input.lower() in ["вихід", "exit", "quit"]:
            print("Дякую за використання Асистента! Гарного дня!")
            break
        try:
            results = process_user_query(user_input)
            print("\n Рекомендовані товари:")
            print(results[["Product_Name", "Calculated_Price"]].to_string(index=False))
            print("\n")
        except Exception as e:
            print(f"Помилка: {e}\n")

# 🔽 Запуск
run_assistant()

Введи 'вихід', щоб завершити.

📝 Ваш запит: Хочу щось до чаю
Помилка: name 'process_user_query' is not defined

